# auto-sklearn

#### Author's description:

auto-sklearn is an automated machine learning toolkit and a drop-in replacement for a scikit-learn estimator

#### Useful links:

[install link](https://automl.github.io/auto-sklearn/master/installation.html),
[git](https://github.com/automl/auto-sklearn),
[manual](https://automl.github.io/auto-sklearn/master/manual.html),
[parallel instances](https://automl.github.io/auto-sklearn/master/examples/example_parallel_manual_spawning.html),
[parallel runs on one machine](https://automl.github.io/auto-sklearn/master/examples/example_parallel_n_jobs.html),
[cross validation](https://automl.github.io/auto-sklearn/master/examples/example_crossvalidation.html),
[feature types](https://automl.github.io/auto-sklearn/master/examples/example_feature_types.html)

#### Usage Note

auto-sklearn builds ensembles, provides good control of the auto-ML run details, and makes exporting easy due to its scikit-learn foundations.

## Install and import

In [1]:
pip install auto-sklearn --user

     |████████████████████████████████| 6.4 MB 5.4 MB/s eta 0:00:01
     |████████████████████████████████| 24.9 MB 93.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 91.5 MB/s eta 0:00:01
     |████████████████████████████████| 890 kB 90.9 MB/s eta 0:00:01
     |████████████████████████████████| 701 kB 86.2 MB/s eta 0:00:01
     |████████████████████████████████| 4.9 MB 83.7 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 98.8 MB/s eta 0:00:01
     |████████████████████████████████| 260 kB 106.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.9 MB 94.4 MB/s eta 0:00:01
     |████████████████████████████████| 141 kB 109.2 MB/s eta 0:00:01
     |████████████████████████████████| 55 kB 11.1 MB/s  eta 0:00:01
     |████████████████████████████████| 284 kB 98.7 MB/s eta 0:00:01
     |████████████████████████████████| 322 kB 105.8 MB/s eta 0:00:01
     |████████████████████████████████| 46 kB 11.0 MB/s eta 0:00:01
  Created wheel for auto-sklearn

In [2]:
# pip install auto-sklearn==0.6.0 --user
"""
original verision of auto-sklearn used
did not capture original versions of other libraries like sklearn
doesn't work on a new compute environment due to version mismatches
best approach is to try on current latest versions of everything
and then freeze all the package versions once it is working again
"""

"\noriginal verision of auto-sklearn used\ndid not capture original versions of other libraries like sklearn\ndoesn't work on a new compute environment due to version mismatches\nbest approach is to try on current latest versions of everything\nand then freeze all the package versions once it is working again\n"

#### *---You may need to restart the kernel after install and run again starting here---*

In [3]:
import autosklearn.classification
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics
import pandas as pd
import numpy as np
import subprocess

In [4]:
#there can be a lot of warnings in auto-sklearn
#especially if you overwrite existing files
#turning off for demo purposes

import warnings
warnings.filterwarnings("ignore")

## Take a look at the classification function

auto-sklearn is mostly a wrapper around scikit-learn. It was not the intention of the authors to allow user control over details such as the modeling algorithm and typical hyper-parameter choices. Control is several layers deep in the [SMAC](https://automl.github.io/SMAC3/master/) space and scenario settings. The user can control the time is takes to build the ensemble, the resampling strategy and the parallelization of the work across CPUs on the machine. These will be demonstrated below.

In [5]:
?autosklearn.classification.AutoSklearnClassifier

## Heart Disease

#### Load the heart disease dataset

Note that in this cell we are calling **sklearn.model_selection.train_test_split()** twice and creating two sets of heart disease (hd) data for model fitting and testing. One is for the hd data without one hot encoding (ohe) and the other has the ohe columns. 

auto-sklearn accepts a list of categorical features and has several methods for treating categorical data. In this notebook we try both approaches - building ohe columns ourselves and letting auto-sklearn do its thing.

In [6]:
'''
/mnt/data/raw/heart.csv

attribute documentation:
      age: age in years
      sex: sex (1 = male; 0 = female)
      cp: chest pain type
        -- Value 1: typical angina
        -- Value 2: atypical angina
        -- Value 3: non-anginal pain
        -- Value 4: asymptomatic
     trestbps: resting blood pressure (in mm Hg on admission to the 
        hospital)
     chol: serum cholestoral in mg/dl
     fbs: (fasting blood sugar > 120 mg/dl)  (1 = true; 0 = false)
     restecg: resting electrocardiographic results
        -- Value 0: normal
        -- Value 1: having ST-T wave abnormality (T wave inversions and/or ST 
                    elevation or depression of > 0.05 mV)
        -- Value 2: showing probable or definite left ventricular hypertrophy
                    by Estes' criteria
     thalach: maximum heart rate achieved
     exang: exercise induced angina (1 = yes; 0 = no)
     oldpeak = ST depression induced by exercise relative to rest
     slope: the slope of the peak exercise ST segment
        -- Value 1: upsloping
        -- Value 2: flat
        -- Value 3: downsloping
     ca: number of major vessels (0-3) colored by flourosopy
     thal: 
         3 = normal; 
         6 = fixed defect; 
         7 = reversable defect
     target: diagnosis of heart disease (angiographic disease status)
        -- Value 0: < 50% diameter narrowing
        -- Value 1: > 50% diameter narrowing
 '''

#load and clean the data----------------------

#column names
names = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang', \
         'oldpeak','slope','ca','thal','target']

#load data from Domino project directory
hd_data = pd.read_csv("./data/raw/heart.csv", header=None, names=names)

#in case some data comes in as string
#convert to numeric and coerce errors to NaN
for col in hd_data.columns:  # Iterate over chosen columns
    hd_data[col] = pd.to_numeric(hd_data[col], errors='coerce')
    
#drop nulls
hd_data.dropna(inplace=True)

#non-ohe data---------------------------------
   
#load the X and y set as a numpy array
X_hd = hd_data.drop('target', axis=1).values
y_hd = hd_data['target'].values

#build the train and test sets
X_hd_train, X_hd_test, y_hd_train, y_hd_test = \
    sklearn.model_selection.train_test_split(X_hd, y_hd, random_state=12)

#now do ohe-----------------------------------

#function to do one hot encoding for categorical columns
def create_dummies(data, cols, drop1st=True):
    for c in cols:
        dummies_df = pd.get_dummies(data[c], prefix=c, drop_first=drop1st)  
        data=pd.concat([data, dummies_df], axis=1)
        data = data.drop([c], axis=1)
    return data

cat_cols = ['cp', 'restecg', 'slope', 'ca', 'thal']
hd_data = create_dummies(hd_data, cat_cols)
    
#load the X and y set as a numpy array
X_hd_ohe = hd_data.drop('target', axis=1).values
y_hd_ohe = hd_data['target'].values

#build the train and test sets
X_hd_ohe_train, X_hd_ohe_test, y_hd_ohe_train, y_hd_ohe_test = \
    sklearn.model_selection.train_test_split(X_hd_ohe, y_hd_ohe, \
                                             random_state=12)

#### Build a model on ohe data with holdout

In [7]:
%%time

#build the auto-sklearn routine
automl_hd_ohe = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=60,
    per_run_time_limit=30,
    disable_evaluator_output=False,
    resampling_strategy='holdout',
    resampling_strategy_arguments={'train_size': 0.67}
)

#call it
automl_hd_ohe.fit(X_hd_ohe_train, y_hd_ohe_train, \
                  dataset_name='heart_disease')

#save the predicitons
predictions_hd_ohe = automl_hd_ohe.predict(X_hd_ohe_test)

[WARNING] [2022-08-11 23:40:38,894:Client-AutoML(1):heart_disease] Capping the per_run_time_limit to 29.0 to have time for a least 2 models in each process.
CPU times: user 10.6 s, sys: 863 ms, total: 11.4 s
Wall time: 58.2 s


#### Fitting with autosklearn

A common mistake is to call **fit_ensemble()** after already running **fit()**. **fit()** both optimizes the machine learning models and builds an ensemble out of them. To disable ensembling when running **fit()** (with parallel instances for example) set ensemble_size to 0. Then **fit_ensemble()** would be needed once all models have been built.

To save fitted models, use typical [pickle procedures](https://scikit-learn.org/stable/modules/model_persistence.html#persistence-example).

#### Metrics

Accuracy, sprint stats, and model details are available. 

Later we will run auto-sklearn in parallel. Note the number of models built here and compare it to the number built with parallelization turned on. 

The model details give you insight into what auto-sklearn is doing under the hood. You can see the modeling algorithm used and all the parameter settings. 

In [8]:
print('Accuracy:')
print(sklearn.metrics.accuracy_score(y_hd_ohe_test, \
                                     predictions_hd_ohe))
print(' ')
print('-----------------------------------------')
print(' ')
print('Sprint Stats:')
print(automl_hd_ohe.sprint_statistics())
print(' ')
print('-----------------------------------------')
print(' ')
print('Model Details:')
print(automl_hd_ohe.show_models())

Accuracy:
0.6973684210526315
 
-----------------------------------------
 
Sprint Stats:
auto-sklearn results:
  Dataset name: heart_disease
  Metric: accuracy
  Best validation score: 0.880000
  Number of target algorithm runs: 25
  Number of successful target algorithm runs: 25
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 0
  Number of target algorithms that exceeded the memory limit: 0

 
-----------------------------------------
 
Model Details:
{24: {'model_id': 24, 'rank': 1, 'cost': 0.12, 'ensemble_weight': 0.14, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f7e5dcd58b0>, 'balancing': Balancing(random_state=1), 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7f7e5dd2a3a0>, 'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x7f7e5dd2a2b0>, 'sklearn_clas

#### Do the same thing (build a model on ohe data with holdout) but this time with parallelization turned on

In [9]:
%%time

#build the auto-sklearn routine
automl_hd_ohe_p = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=60,
    per_run_time_limit=30,
    disable_evaluator_output=False,
    resampling_strategy='holdout',
    resampling_strategy_arguments={'train_size': 0.67},
    
    #turn on parallelization
    n_jobs=4,
    seed=5,
)

#call it
automl_hd_ohe_p.fit(X_hd_ohe_train, y_hd_ohe_train, \
                    dataset_name='heart_disease')

#save the predicitons
predictions_hd_ohe_p = automl_hd_ohe_p.predict(X_hd_ohe_test)

[WARNING] [2022-08-11 23:41:38,130:Client-AutoML(5):heart_disease] Capping the per_run_time_limit to 29.0 to have time for a least 2 models in each process.
[ERROR] [2022-08-11 23:42:35,860:asyncio.events] 
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.8/site-packages/distributed/utils.py", line 799, in wrapper
    return await func(*args, **kwargs)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/distributed/client.py", line 1246, in _reconnect
    await self._ensure_connected(timeout=timeout)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/distributed/client.py", line 1276, in _ensure_connected
    comm = await connect(
  File "/home/ubuntu/.local/lib/python3.8/site-packages/distributed/comm/core.py", line 315, in connect
    await asyncio.sleep(backoff)
  File "/opt/conda/lib/python3.8/asyncio/tasks.py", line 659, in sleep
    return await future
asyncio.exceptions.CancelledError
[ERROR] [2022-08-11 23:42:35,862:asyncio.events] 
Traceback 

In [10]:
print('Accuracy:')
print(sklearn.metrics.accuracy_score(y_hd_ohe_test, \
                                     predictions_hd_ohe_p))
print(' ')
print('-----------------------------------------')
print(' ')
print('Sprint Stats:')
print(automl_hd_ohe_p.sprint_statistics())

Accuracy:
0.6973684210526315
 
-----------------------------------------
 
Sprint Stats:
auto-sklearn results:
  Dataset name: heart_disease
  Metric: accuracy
  Best validation score: 0.893333
  Number of target algorithm runs: 34
  Number of successful target algorithm runs: 34
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 0
  Number of target algorithms that exceeded the memory limit: 0



In [11]:
print('Model Details:')
print(automl_hd_ohe_p.show_models())

Model Details:
{24: {'model_id': 24, 'rank': 1, 'cost': 0.10666666666666669, 'ensemble_weight': 0.34, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f7e4e740af0>, 'balancing': Balancing(random_state=5), 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7f7e4e7a05b0>, 'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x7f7e4e7a0130>, 'sklearn_classifier': AdaBoostClassifier(algorithm='SAMME',
                   base_estimator=DecisionTreeClassifier(max_depth=2),
                   learning_rate=0.13167493237005792, n_estimators=56,
                   random_state=5)}, 13: {'model_id': 13, 'rank': 2, 'cost': 0.1333333333333333, 'ensemble_weight': 0.06, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f7e4e7b5e80>, 'balancing': Balancing(random_state=5), 'feature_p

## Breast Cancer

#### Load the breast cancer data

In [12]:
from sklearn.datasets import load_breast_cancer
print(sklearn.datasets.load_breast_cancer()['DESCR'])

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

In [13]:
#load from sklearn
X_bc, y_bc = sklearn.datasets.load_breast_cancer(return_X_y=True)

#build the train and test sets
X_bc_train, X_bc_test, y_bc_train, y_bc_test = \
    sklearn.model_selection.train_test_split(X_bc, y_bc, random_state=1)

#### Build a model using holdout and parallelization

In [14]:
%%time

#build the auto-sklearn routine
automl_bc = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=60,
    per_run_time_limit=30,
    disable_evaluator_output=False,
    # 'holdout' with 'train_size'=0.67 is the default argument setting
    # for AutoSklearnClassifier. It is explicitly specified in this example
    # for demonstrational purpose.
    resampling_strategy='holdout',
    resampling_strategy_arguments={'train_size': 0.67},
    n_jobs=4,
    seed=5,
)

#call it
automl_bc.fit(X_bc_train, y_bc_train, dataset_name='breast_cancer')

#save the predicitons
predictions_bc = automl_bc.predict(X_bc_test)

[WARNING] [2022-08-11 23:42:41,972:Client-AutoML(5):breast_cancer] Capping the per_run_time_limit to 29.0 to have time for a least 2 models in each process.
[ERROR] [2022-08-11 23:43:40,528:asyncio.events] 
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.8/site-packages/distributed/utils.py", line 799, in wrapper
    return await func(*args, **kwargs)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/distributed/client.py", line 1246, in _reconnect
    await self._ensure_connected(timeout=timeout)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/distributed/client.py", line 1276, in _ensure_connected
    comm = await connect(
  File "/home/ubuntu/.local/lib/python3.8/site-packages/distributed/comm/core.py", line 315, in connect
    await asyncio.sleep(backoff)
  File "/opt/conda/lib/python3.8/asyncio/tasks.py", line 659, in sleep
    return await future
asyncio.exceptions.CancelledError
[ERROR] [2022-08-11 23:43:40,529:asyncio.events] 
Traceback 

In [15]:
print('Accuracy:')
print(sklearn.metrics.accuracy_score(y_bc_test, \
                                     predictions_bc))
print(' ')
print('-----------------------------------------')
print(' ')
print('Sprint Stats:')
print(automl_bc.sprint_statistics())

Accuracy:
0.951048951048951
 
-----------------------------------------
 
Sprint Stats:
auto-sklearn results:
  Dataset name: breast_cancer
  Metric: accuracy
  Best validation score: 0.985816
  Number of target algorithm runs: 40
  Number of successful target algorithm runs: 40
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 0
  Number of target algorithms that exceeded the memory limit: 0



In [16]:
print('Model Details:')
print(automl_bc.show_models())

Model Details:
{3: {'model_id': 3, 'rank': 1, 'cost': 0.014184397163120588, 'ensemble_weight': 0.1, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f7e4e965880>, 'balancing': Balancing(random_state=5), 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7f7e50f94cd0>, 'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x7f7e50f94310>, 'sklearn_classifier': MLPClassifier(activation='tanh', alpha=0.0001363185819149026, beta_1=0.999,
              beta_2=0.9, early_stopping=True,
              hidden_layer_sizes=(115, 115, 115),
              learning_rate_init=0.00018009776276177523, max_iter=32,
              n_iter_no_change=32, random_state=5, verbose=0, warm_start=True)}, 7: {'model_id': 7, 'rank': 2, 'cost': 0.014184397163120588, 'ensemble_weight': 0.12, 'data_preprocessor': <autosklearn.pipeline.components.data_preproces

## Model Run and Accuracy Stats

All in one place for easier comparison.

In [17]:
print("-----------Heart Disease---------------")
print(' ')
print(' ')

print("Model stats HD Holdout:")
print(automl_hd_ohe.sprint_statistics())
print(' ')
print("Accuracy score HD Holdout:")
print(sklearn.metrics.accuracy_score(y_hd_ohe_test, \
                                     predictions_hd_ohe))

print(' ')
print('-----------------------------------------')
print(' ')

print("Model stats HD Holdout Parallel:")
print(automl_hd_ohe_p.sprint_statistics())
print("Accuracy score HD Holdout Parallel:")
print(sklearn.metrics.accuracy_score(y_hd_ohe_test, \
                                     predictions_hd_ohe_p))

print(' ')
print('-----------------------------------------')
print(' ')

# #holdout parallel feat_type
# print("Model stats HD Holdout Feature Type Parallel:")
# print(automl_hd_ft_p.sprint_statistics())
# print(' ')
# print("Accuracy score HD Holdout Feature Type Parallel:")
# print(sklearn.metrics.accuracy_score(y_hd_test, \
#                                      predictions_hd_ft_p))

# print(' ')
# print('-----------------------------------------')
# print(' ')

# #cross validation parallel
# print("Model stats HD CV Parllel:")
# print(automl_hd_cv_p.sprint_statistics())
# print(' ')
# print("Accuracy score HD CV Parallel:")
# print(sklearn.metrics.accuracy_score(y_hd_ohe_test, \
#                                      predictions_hd_cv_p))

print(' ')
print(' ')

print("-----------Breast Cancer---------------")
print(' ')
print(' ')

print("Model stats BC Holdout Parallel:")
print(automl_bc.sprint_statistics())
print(' ')
print("Accuracy score BC Holdout Parallel:")
print(sklearn.metrics.accuracy_score(y_bc_test, \
                                     predictions_bc))

-----------Heart Disease---------------
 
 
Model stats HD Holdout:
auto-sklearn results:
  Dataset name: heart_disease
  Metric: accuracy
  Best validation score: 0.880000
  Number of target algorithm runs: 25
  Number of successful target algorithm runs: 25
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 0
  Number of target algorithms that exceeded the memory limit: 0

 
Accuracy score HD Holdout:
0.6973684210526315
 
-----------------------------------------
 
Model stats HD Holdout Parallel:
auto-sklearn results:
  Dataset name: heart_disease
  Metric: accuracy
  Best validation score: 0.893333
  Number of target algorithm runs: 34
  Number of successful target algorithm runs: 34
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 0
  Number of target algorithms that exceeded the memory limit: 0

Accuracy score HD Holdout Parallel:
0.6973684210526315
 
--------------------

## Save Ensemble to Disk

In the specific case of [scikit-learn](https://scikit-learn.org/stable/modules/model_persistence.html#persistence-example), it may be better to use joblib’s replacement of pickle (dump & load), which is more efficient on objects that carry large numpy arrays internally as is often the case for fitted scikit-learn estimators, but can only pickle to the disk and not to a string.

In [18]:
from joblib import dump, load
dump(automl_bc, 'autosklearn_bc.joblib') 

['autosklearn_bc.joblib']

## Save to Domino Stats File

To keep things simple, we pick one of the hd models. Saving stats to this file [allows Domino to track and trend them in the Experiment Manager](https://support.dominodatalab.com/hc/en-us/articles/204348169-Diagnostic-statistics-with-dominostats-json) when this notebook is run as a batch or scheduled job.

In [19]:
hd_acc = sklearn.metrics.accuracy_score(y_hd_ohe_test, \
                                        predictions_hd_ohe_p)
bc_acc = sklearn.metrics.accuracy_score(y_bc_test, \
                                        predictions_bc)

import json
with open('dominostats.json', 'w') as f:
    f.write(json.dumps( {"HD_ACC": hd_acc, "BC_ACC": bc_acc}))